# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import nltk
nltk.download(['punkt','wordnet','averaged_perceptron_tagger','stopwords'])

import re
import os
import numpy as np
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load data from database


engine = create_engine('sqlite:///STOTAJO.db')
df = pd.read_sql_table("MESSAGE_CATEGORIES", engine)
print(df.head())
X = df['message'].values
Y = df[df.columns[4:]]

print(Y.columns.values)

   id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products      ...        \
0        0      0            0             0                 0      ...         
1        0      0            1        

### 2. Write a tokenization function to process your text data

In [4]:
url_regex='http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    """
    
    
    """
    detected_urls=re.findall(url_regex,text)
    for character in string.punctuation:
        text=text.replace(character,'')
    words=word_tokenize(text)
    tokens=[w for w in words if w not in stopwords.words ("english")]
    lemmatizer=WordNetLemmatizer()
    
    clean_tokens=[]
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens
    
    pass




### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline ([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(max_depth=6)))
    ])

pipeline.get_params()


{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fa9e0c21ea0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=6, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, ve

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test=train_test_split(X,Y)

pipeline.fit(X_train,y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred=pipeline.predict(X_train)


In [7]:
#print(classification_report(y_train,y_pred,target_names = df.columns[4:]))
#print(classification_report(np.hstack(y_test),np.hstack(y_pred)))

print(classification_report(y_train, y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87     15059
               request       0.00      0.00      0.00      3401
                 offer       0.00      0.00      0.00        90
           aid_related       0.91      0.07      0.13      8162
          medical_help       1.00      0.00      0.00      1588
      medical_products       0.00      0.00      0.00       999
     search_and_rescue       0.00      0.00      0.00       554
              security       0.00      0.00      0.00       342
              military       0.00      0.00      0.00       626
                 water       0.00      0.00      0.00      1281
                  food       1.00      0.00      0.00      2220
               shelter       1.00      0.00      0.00      1731
              clothing       0.00      0.00      0.00       290
                 money       0.00      0.00      0.00       448
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {
      'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_estimators': [10, 20] ,
         'clf__estimator__min_samples_split': [2, 4, 6]
    
    
}

cv = GridSearchCV(pipeline,param_grid=parameters,n_jobs=-1,cv=2)

cv.fit(X_train,y_train)

print("\nBest Parameters:",cv.best_params_)


Best Parameters: {'clf__estimator__min_samples_split': 6, 'clf__estimator__n_estimators': 20, 'vect__ngram_range': (1, 2)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
y_pred=cv.predict(X_test)

accuracy=(y_pred==y_test).mean()

print(classification_report(y_test, y_pred, target_names=Y.columns.values))
print(accuracy)


                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5034
               request       0.00      0.00      0.00      1073
                 offer       0.00      0.00      0.00        28
           aid_related       1.00      0.01      0.02      2698
          medical_help       0.00      0.00      0.00       496
      medical_products       0.00      0.00      0.00       314
     search_and_rescue       0.00      0.00      0.00       170
              security       0.00      0.00      0.00       129
              military       0.00      0.00      0.00       234
                 water       0.00      0.00      0.00       391
                  food       0.00      0.00      0.00       703
               shelter       0.00      0.00      0.00       583
              clothing       0.00      0.00      0.00       115
                 money       0.00      0.00      0.00       156
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
pipeline2 = Pipeline ([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
    ])

print(pipeline2.get_params())


{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7fa9e0c21ea0>, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))], 'vect': CountVectorizer(analyzer='word', binary=

In [ ]:

parameters2 = {
      'vect__ngram_range': ((1, 1), (1, 2)), 
      'clf__estimator__min_samples_leaf': [1, 10]
}

pipeline2.fit(X_train,y_train)


cv2 = GridSearchCV(pipeline2,param_grid=parameters2,n_jobs=-1,cv=2)

cv2.fit(X_train,y_train)

print("\nBest Parameters:",cv2.best_params_)


In [23]:
y_pred=cv2.predict(X_test)

accuracy=(y_pred==y_test).mean()

print(classification_report(y_test, y_pred, target_names=Y.columns.values))
print(accuracy)

                        precision    recall  f1-score   support

               related       0.85      0.94      0.89      5034
               request       0.78      0.57      0.66      1073
                 offer       0.00      0.00      0.00        28
           aid_related       0.76      0.70      0.73      2698
          medical_help       0.66      0.27      0.38       496
      medical_products       0.70      0.26      0.38       314
     search_and_rescue       0.68      0.20      0.31       170
              security       0.00      0.00      0.00       129
              military       0.57      0.27      0.37       234
                 water       0.82      0.52      0.63       391
                  food       0.84      0.66      0.74       703
               shelter       0.78      0.48      0.60       583
              clothing       0.59      0.35      0.44       115
                 money       0.61      0.21      0.31       156
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [24]:
import pickle

pickle.dump(cv2, open('classifier.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.